### Ćelijski automati

Ćelijski automati su klasa modela koji se mogu koristiti za simulaciju različitih prirodnih fenomena. Korišćeni su za modeliranje razvoja tumora, kroz trodimenzionalnu mapu tkiva i predviđanje rasta ćelija kroz vreme. U neuronauci, ćelijski automati se mogu koristi za izučavanje populacije aktivacije neurona. Postoje mnogo druge primene, uključujući one u hemiji, biologiji, ekologiji i mnogim drugim granama nauke.

Ćelijski automati su u osnovi skup (prostorno) uređenih jedinica koje nazivamo ćelije. Svaka ćelija vrši neku od mogućih akcija, ili uzima neko od mogućih stanja, u zavisnosti od svoje neposredne okoline. Možda najpoznatiji primer ćelijskih automata je Konvejeva Igra života.

### Igra života

Konvejeva Igra života je ćelijski automat koji se sastoji iz pravougaone mreže ćelija. Svaka ćeilja može biti u jednom od sva stanja: *živa* i *mrtva*. Igra se odvija kroz vremenske iteracije, gde ćelije u jednoj iteraciji računaju svoje stanje u sledećoj. Stanje ćelije se računa na osnovu svog trenutnog stanja, trenutnog stanja svih neposrednih 8 suseda ćelije i sledećih pravila:
- Ako ćelija ima manje od 2 živa suseda, u sledećoj iteraciji će biti mrtva 
- Ako ćelija ima više od 3 živa suseda, u sledećoj iteraciji će biti mrtva 
- Ako je ćelija živa i ima 2 ili 3 živa suseda, u sledećoj iteraciji će biti živa
- Ako je ćelija mrtva i ima 3 živa suseda, u sledećoj iteraciji će biti živa

Osnovna struktura podataka Igre života je matrica stanja, koja za svaku ćeliju a[i, j] sadrži podatak da li je ćelija živa ili mrtva. Pri implementaciji igre javlja se problem suseda ćelija koje se nalaze uz ivicu mreže (ćelije u prvoj koloni nemaju "leve" susede, itd.). Postoji više rešenja za ovaj problem kao što su:
- Pretpostaviti da su nepostojeći susedi uvek mrtvi
- Uvesti "cikluse", tako da je poslednja kolona "levi" sused prve kolone, a poslednji red "gornji" sused prvog reda (efektivno torus).

Pri izradi zadataka koristiti drugi pristup.

### Zadatak
Zadatak je implementirati paralelizovanu verziju Igre života u programskom jeziku Python, na nekoliko načina. Prilikom implementacije obezbediti da se posle izvršavanja zadatog broja iteracija sačuva niz matrica stanja kroz vreme (stanja sistema u svakoj iteraciji), koji je kompatibilan sa datom funkcijom za animaciju (sledeća ćelija u fajlu).

1. Upotrebom niti koje simuliraju *po jednu ćeliju* i sinhronizacijom Ključevima, Semaforima i Uslovima (7 poena)  
  Svaka nit simulira rad jedne ćelije u sistemu i ima pristup stanjima svojih suseda. Ćelije nemaju pristup globalnom brojaču iteracija, već svaka ćelija interno vodi računa o broju trenutne iteracije. Pored matrice podataka potrebno je uvesti:
  - Listu brojača suseda koji su pročitali trenutnu vrednost (za svaku ćeliju po jedan brojač). Osmi (poslednji) sused koji pročita vrednost budi ćeliju kako bi mogla da upiše novu vrednost u matricu stanja (buđenje realizovati semaforom). Voditi računa o sinhronizaciji suseda koji menjaju vrednost brojača.
  - Uslov (Condition) na kome čekaju sve ćelije koje su upisale novu vrednost u matricu stanja, pre nego što pređu u sledeću iteraciju. 
  - Brojač ćelija, zaštićen ključem, koje su upisale novu vrednost u matricu stanja. Poslednja ćelija aktivira Uslov za sledeću iteraciju. Voditi računa o redosledu uzimanja i puštanja ključa za brojač i ključa za uslov.
  
2. Upotrebom niti koje simuliraju *po jednu ćeliju* i sinhronizacijom redovima za poruke (6 poena)  
  Svaka nit simulira rad jedne ćelije u sistemu. Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). Ćelije podatke o svojem stanju razmenjuju putem reda za poruke. Za potrebe analize rada može se uvesti deljeni niz matrica stanja (i-ti element niza je matrica stanja i-te iteracije), u koji ćelije upisuju svoja stanja (ćelije ne mogu čitati iz ovog niza!).
  
3. Upotrebom procesa koji simuliraju *po jednu ćeliju* i sinhronizacijom redovima za poruke (6 poena)  
  Svaki proces je simulira rad jedne ćelije u sistemu. Stanje svake ćelije se čuva unutar ćelije (rad sistema se ne oslanja na deljenu matricu stanja). Ćelije podatke o svojem stanju razmenjuju putem reda za poruke. Za potrebe analize rada implementirati poseban servis (dodatni proces) kojem sve ćelije javljaju novo stanje prilikom promene (pri čemu poruke sadrže koordinate ćelije, broj iteracije i novo stanje). Servis treba da rekonstruiše i sačuva (ili vrati u glavni program) niz matrica stanja.

4. Upotrebom više procesa kroz *process pool*, generisanjem taskova na nivou skupa ćelija (6 poena)  
  Matricu stanja podeliti na N delova (gde je konfigurabilni parametar) i za svaki deo generisati *task* (poziv funkcije čijim parametrima se definiše koji deo matrice treba obraditi). Funkcija treba da vrati niz koordinata ćelija i njihove nove vrednosti, a matrica za sledeću iteraciju se može kreirati u glavnom programu. Trenutne vrednosti ćelija i suseda se mogu čitati iz deljene matrice.
  
***Napomena:*** Zadati pristupi 1-3 nisu uobičajen način paralelizacije Igre života, ali su odabrani kao dobar primer za vežbu rada sa paralelnim modelima i tehnikama sinhronizacije.

In [ ]:
# PRVA TACKA
import threading
import time
import numpy as np
import sys

allCellsSetCondition = threading.Condition()
counterLock = threading.Lock()
matrixOfSemaphores = None

currIterMatrix = None
nextIterMatrix = None
listOfMatrix = []
matrixOfCounters = None
setCellsCounter = 0

def resetMatrix(n):
    global matrixOfCounters
    for row in range(n):
        for col in range(n):
            matrixOfCounters[row, col] = 0


def checkIfAllCellsAreSet(n):
    '''Funkcija koja proverava da li su sve celije postavile polja u tekucoj iteraciji'''
    global setCellsCounter
    global listOfMatrix
    global nextIterMatrix
    global currIterMatrix
    counterLock.acquire()
    if setCellsCounter == n**2:
        setCellsCounter = 0
        counterLock.release()
        toAdd = nextIterMatrix.copy()
        listOfMatrix.append(toAdd)
        currIterMatrix = nextIterMatrix.copy()
        allCellsSetCondition.acquire()
        allCellsSetCondition.notifyAll()
        allCellsSetCondition.release()   
        resetMatrix(n)    
        return True
    else:
        counterLock.release()
        return False


def updateCell(row, col, n, iterations):
    ''' Semafor se koristi da zakljuca matricu kada se upisuje u njoj
        Kljuc se koristi da zakljuca brojac koji broji koliko su polja upisana
        Uslov se koristi da bi presli u narednu iteraciju kada sve nitu upisu svoja nova polja '''
    global setCellsCounter
    global nextIterMatrix
    global currIterMatrix 
    global matrixOfCounters
    global matrixOfSemaphores
    for iter in range(iterations):
        alive = 0
        for neighbourRow in range(3):
            for neighbourCol in range(3):
                if neighbourRow == 1 and neighbourCol == 1:
                    continue;
                curr = row + neighbourRow - 1
                currNeighbourRow = ((curr % n) + n) % n
                curr = col + neighbourCol - 1
                currNeightbourCol = ((curr % n) + n) % n
                counterLock.acquire()
                matrixOfCounters[currNeighbourRow, currNeightbourCol] += 1
                if matrixOfCounters[currNeighbourRow, currNeightbourCol] == 8:
                    matrixOfSemaphores[currNeighbourRow][currNeightbourCol].release()
                counterLock.release()
                if currIterMatrix[currNeighbourRow, currNeightbourCol] == 1:
                    alive += 1
        newValue = 0
        if alive < 2 or alive > 3:
            newValue  = 0
        if currIterMatrix[row, col] == 1 and 2 <= alive <= 3:
            newValue  = 1
        # TODO: Proveriti pravilo kada je celija mrtva i ima 2 suseda
        if currIterMatrix[row, col] == 0 and alive == 3:
            newValue  = 1
        if matrixOfCounters[row, col] != 8:
            matrixOfSemaphores[row][col].acquire()
        nextIterMatrix[row, col] = newValue
        counterLock.acquire()
        setCellsCounter += 1
        counterLock.release()
        allCellsSetCondition.acquire()
        if not checkIfAllCellsAreSet(n):    
            allCellsSetCondition.wait()
        allCellsSetCondition.release()


def parallel(n, iterations):
    global listOfMatrix
    global nextIterMatrix
    global currIterMatrix
    global matrixOfCounters  
    global matrixOfSemaphores
    listOfMatrix.clear()
    nextIterMatrix = np.zeros((n, n))
    matrixOfCounters = np.zeros((n, n))   
    defineRow = [threading.Semaphore(0) for i in range(n)]  
    matrixOfSemaphores = [list(defineRow) for i in range(n)]
    threadArray = []
    currIterMatrix = (np.random.rand(n ** 2).reshape(n, n) > 0.5).astype(np.int8)
    listOfMatrix.append(currIterMatrix)  
    for row in range(n):
        for col in range(n):
            currThread = threading.Thread(target=updateCell, args=(row, col, n, iterations))
            threadArray.append(currThread)
    for thread in threadArray:
        thread.start()
    for thread in threadArray:
        thread.join()
    return listOfMatrix

In [ ]:
from matplotlib.animation import FuncAnimation
import matplotlib.pyplot as plt
from IPython.display import HTML

def animate(steps):
  ''' Prima niz matrica (svaka matrica je stanje u jednom koraku simulacije) 
  prikazuje razvoj sistema'''
  
  def init():
    im.set_data(steps[0])
    return [im]
  
  
  def animate(i):
    im.set_data(steps[i])
    return [im]

  im = plt.matshow(steps[0], interpolation='None', animated=True);
  
  anim = FuncAnimation(im.get_figure(), animate, init_func=init,
                  frames=len(steps), interval=500, blit=True, repeat=False);
  return anim


n = 20
iterations = 20
steps = parallel(n, iterations)
anim = animate(steps)
print(1)
HTML(anim.to_html5_video())